## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "./constantes.ipynb"

#b. Constantes a definir por el usuario.
#i. Dataset Train.
dataset_train = datasets_path + "competencia_02_stacking.parquet"
#ii. Dataset Test.
dataset_test = datasets_path + "competencia_02_stacking_test.parquet"
#iii. Información sobre rango temporal del modelo.
cantidad_meses_train = 12
#iv. Ruta del modelo guardado post-optimización de hiperparámetros.
modelo =  modelos_path + 'STACKING_{}m.txt'.format(cantidad_meses_train)

In [ ]:
#3. Lectura de datos.
#i. Entrenamiento.
train_data = pd.read_parquet(dataset_train)
#ii. Test.
test_data = pd.read_parquet(dataset_test)

In [ ]:
#4. Obtenemos los hiperparámetros del modelo.
model_lgb = lgb.Booster(model_file=modelo)
parametros = model_lgb.params # Parámetros.
best_iter = parametros["num_iterations"]  # Mejor número de iteraciones guardado en el modelo.

In [ ]:
#5. Pesos y reclusterización.
#i. Train.
train_data['clase_peso'] = 1.0

train_data.loc[train_data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
train_data.loc[train_data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

train_data['clase_binaria2'] = 0
train_data['clase_binaria2'] = np.where(train_data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#6. Eliminamos numero_de_cliente y foto_mes del análisis.
#a. Train.
train_data.drop(["numero_de_cliente","foto_mes"],axis=1,inplace=True)
#b. Test.
test_data.drop(["numero_de_cliente","foto_mes"],axis=1,inplace=True)

In [ ]:
#7. Dividimos los datasets en las columnas de interés.
#a. Datos para entrenar todo el modelo final para Kaggle.
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

## B. Envíos para encontrar el punto de corte óptimo con 10 semillas.

In [ ]:
#1. Configuramos la cantidad de veces que vamos a subir cada curva de intentos (semillas).
cantidad_semillas = 10
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Entrenamos y predecimos Agosto.
#i. Recorremos diferentes semillas para entrenar.
i= 0
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla: {}.".format(semilla))
    #ii. Activamos la API para realizar envíos a Kaggle.
    api = KaggleApi()
    api.authenticate()
    #iii. Tomamos el mejor modelo y con eso entrenamos.
    print("--> Mejores parámetros...")
    params = parametros.copy()  
    params['seed'] = semilla
    #iv. Adaptamos el dataset de entrenamiento.
    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)
    #v. Entrenamos el modelo de Light GBM con los mejores hiperparámetros y la semilla aleatoria.
    print("--> Comienza el entrenamiento...")
    model_lgb = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    #vi. Predecimos Agosto.
    predicciones = model_lgb.predict(test_data)
    #vii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    test_data['Probabilidad'] = predicciones
    #viii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
    tb_entrega = test_data.sort_values(by='Probabilidad', ascending=False)
    #ix. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
    cortes = range(8500,13500,500)
    #x. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
    num_subida_kaggle = 1
    print("--> Comienzan los envíos a Kaggle...\n")
    for envios in cortes:
        #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
        tb_entrega['Predicted'] = 0
        tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
        resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
        
        print("Cantidad de clientes {}".format(envios))
        #2. Guardamos el archivo para Kaggle.
        nombre_archivo = "KSt_{}_EK_0{}_{}.csv".format(cantidad_meses_train,num_subida_kaggle,semilla) #DF con -X meses, número de intento, número de semilla.
        ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
        resultados.to_csv(ruta_archivo, index=False)
        
        num_subida_kaggle += 1
        i += 1
        
        #3. Envío a Kaggle.
        #a. Defino los parámetros claves.
        mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train}. Semilla:{semilla},corte:{envios}' # Datos de Train, semilla y punto de corte de prueba.
        competencia = 'dm-ey-f-2024-segunda'
        #c. Subo la Submission.
        while i<=18:
                api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
                print("Submission successful!")
                break
        else:
            print("Esperamos 30 segundos...")
            time.sleep(30)
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            i= 0
        
    #d. Borramos la columna de predicciones anterior.
    test_data.drop(["Probabilidad"],axis=1,inplace=True)

In [ ]:
#3. Análisis de las subidas a Kaggle.
#i. Eligiendo el mejor punto de corte.
# Dados las 10 semillas corridas, y los 10 diferentes puntos de corte, el promedio nos indica que el punto de corte
# más óptimo para este modelo es X envíos. En tanto la distribución promedio del Público se asemeja a la
# distribución promedio del Privado, dicho punto de corte sería el mejor para la distribución promedio del Privado.

## C. Envios a Kaggle para elegir luego el valor Promedio del punto de corte óptimo (estabilizar el modelo).

In [ ]:
#1. Configuramos la cantidad de veces que vamos a subir cada curva de intentos (semillas).
cantidad_semillas = 40
semillas = [np.random.randint(0, 10000) for _ in range(cantidad_semillas)]

In [ ]:
#2. Entrenamos y predecimos Agosto.
#i. Recorremos diferentes semillas para entrenar.
i= 0
for semilla in semillas:
    print("\n---------------------------------------")
    print("Semilla: {}.".format(semilla))
    #ii. Activamos la API para realizar envíos a Kaggle.
    api = KaggleApi()
    api.authenticate()
    #iii. Tomamos el mejor modelo y con eso entrenamos.
    print("--> Mejores parámetros...")
    params = parametros.copy() 
    params['seed'] = semilla
    #iv. Adaptamos el dataset de entrenamiento.
    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)
    #v. Entrenamos el modelo de Light GBM con los mejores hiperparámetros y la semilla aleatoria.
    print("--> Comienza el entrenamiento...")
    model_lgb = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    #vi. Predecimos Agosto.
    predicciones = model_lgb.predict(X_test)
    #vii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
    X_test['Probabilidad'] = predicciones
    #viii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
    tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
    #ix. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
    cortes = range(8500,13500,500)
    #x. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
    num_subida_kaggle = 1
    print("--> Comienzan los envíos a Kaggle...\n")
    for envios in cortes:
        #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
        tb_entrega['Predicted'] = 0
        tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
        resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
        
        print("Cantidad de clientes {}".format(envios))
        #2. Guardamos el archivo para Kaggle.
        nombre_archivo = "KSt_{}_EK_0{}_{}.csv".format(cantidad_meses_train,num_subida_kaggle,semilla) #DF con -X meses, número de intento, número de semilla.
        ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
        resultados.to_csv(ruta_archivo, index=False)
        
        num_subida_kaggle += 1
        i += 1
        
        #3. Envío a Kaggle.
        #a. Defino los parámetros claves.
        mensaje = f'Archivo {nombre_archivo}.Train {cantidad_meses_train}. Semilla:{semilla},corte:{envios}' # Datos de Train, semilla y punto de corte de prueba.
        competencia = 'dm-ey-f-2024-segunda'
        #c. Subo la Submission.
        while i<=18:
                api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
                print("Submission successful!")
                break
        else:
            print("Esperamos 30 segundos...")
            time.sleep(30)
            api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
            print("Submission successful!")
            i= 0
        
    #d. Borramos la columna de predicciones anterior.
    X_test.drop(["Probabilidad"],axis=1,inplace=True)